In [503]:

# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
        self.neurons = None
        self.weights = None
        self.bias = None

    def get_weights(self):
        weights = np.vstack([neuron.weights for neuron in self.neurons])
        return weights.T

    def get_bias(self):
        bias = np.vstack([neuron.bias for neuron in self.neurons])
        return bias.T

    def set_weights(self, weights, update_layer=True):
        for neuron, weight in zip(self.neurons, weights):
            neuron.weights = weight
        if (update_layer == True):
            self.weights = self.get_weights()

    def set_bias(self, bias, update_layer=True):
        for neuron, b in zip(self.neurons, bias):
            neuron.bias = b
        if (update_layer == True):
            self.bias = self.get_bias()

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [504]:
import numpy as np

class Neuron():
    def __init__(self, input_dim):
        self.weights = np.random.rand(input_dim)
        self.bias = np.random.random()

    def summing_junction(self):
        uk = np.dot(self.input, self.weights)
        return uk + self.bias
    
    def forward_propagation(self, input_signal):
        self.input = input_signal
        self.output = self.summing_junction()
        #self.output = self.activation(vk)
        return self.output

In [505]:
# inherit from base class Layer
class DenseLayer(Layer):
    # input_dim = number of input neurons
    # output_dim = number of output neurons
    def __init__(self, input_dim, output_dim):
        self.neurons = [Neuron(input_dim) for _ in range(output_dim)]
        self.weights = self.get_weights()
        self.bias = self.get_bias()

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
#        self.output = np.vstack([neuron.forward_propagation(input_data) for neuron in self.neurons]).T
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [506]:


# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [507]:

# activation function and its derivative
def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

In [508]:

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [509]:

class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)
        return layer

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [510]:
# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

weights1 = [[0.95, 0.96],
            [0.8, 0.85]]
bias1 = [0.2, 0.1]

weights2 = [[0.9, 0.8]]
bias2 = [0.3461]

# network
net = Network()
hidden_layer = net.add(DenseLayer(2, 2))
hidden_layer.set_weights(weights1)
hidden_layer.set_bias(bias1)
net.add(ActivationLayer(tanh, tanh_prime))

output_layer = net.add(DenseLayer(2, 1))
output_layer.set_weights(weights2)
output_layer.set_bias(bias2)
net.add(ActivationLayer(tanh, tanh_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

epoch 1/1000   error=0.304569
epoch 2/1000   error=0.266801
epoch 3/1000   error=0.242755
epoch 4/1000   error=0.232443
epoch 5/1000   error=0.228815
epoch 6/1000   error=0.227356
epoch 7/1000   error=0.226493
epoch 8/1000   error=0.225788
epoch 9/1000   error=0.225127
epoch 10/1000   error=0.224487
epoch 11/1000   error=0.223863
epoch 12/1000   error=0.223258
epoch 13/1000   error=0.222676
epoch 14/1000   error=0.222121
epoch 15/1000   error=0.221597
epoch 16/1000   error=0.221104
epoch 17/1000   error=0.220646
epoch 18/1000   error=0.220222
epoch 19/1000   error=0.219833
epoch 20/1000   error=0.219477
epoch 21/1000   error=0.219153
epoch 22/1000   error=0.218859
epoch 23/1000   error=0.218592
epoch 24/1000   error=0.218349
epoch 25/1000   error=0.218128
epoch 26/1000   error=0.217924
epoch 27/1000   error=0.217736
epoch 28/1000   error=0.217561
epoch 29/1000   error=0.217395
epoch 30/1000   error=0.217238
epoch 31/1000   error=0.217086
epoch 32/1000   error=0.216939
epoch 33/1000   e

epoch 236/1000   error=0.199624
epoch 237/1000   error=0.199592
epoch 238/1000   error=0.199560
epoch 239/1000   error=0.199528
epoch 240/1000   error=0.199497
epoch 241/1000   error=0.199465
epoch 242/1000   error=0.199434
epoch 243/1000   error=0.199403
epoch 244/1000   error=0.199373
epoch 245/1000   error=0.199342
epoch 246/1000   error=0.199312
epoch 247/1000   error=0.199281
epoch 248/1000   error=0.199251
epoch 249/1000   error=0.199222
epoch 250/1000   error=0.199192
epoch 251/1000   error=0.199163
epoch 252/1000   error=0.199133
epoch 253/1000   error=0.199104
epoch 254/1000   error=0.199076
epoch 255/1000   error=0.199047
epoch 256/1000   error=0.199018
epoch 257/1000   error=0.198990
epoch 258/1000   error=0.198962
epoch 259/1000   error=0.198934
epoch 260/1000   error=0.198906
epoch 261/1000   error=0.198878
epoch 262/1000   error=0.198851
epoch 263/1000   error=0.198824
epoch 264/1000   error=0.198796
epoch 265/1000   error=0.198769
epoch 266/1000   error=0.198743
epoch 26